In [1]:
import os

import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()
# db = FAISS.from_texts(texts=['9'], embedding=embeddings)
# db.save_local(folder_path=index_path, index_name=index_name) 
# db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [3]:
def gpt_system_user(
    system_message: str, user_message: str, model: str = "gpt-3.5-turbo"
):
    """Para usar en notebooks"""
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        messages = [
                        {"content": system_message, "role": "system"},
                        {"content": user_message, "role": "user"},
                    ]
        functions = [ 
                        {
                          "name": "create_appointment",
                          "description": "Create a medical appointment",
                          "parameters": {
                              "type": "object",
                              "properties": {
                                  "patient_name": {
                                      "type": "string",
                                      "description": "Name of the patient",
                                  },
                                  "patient_email": {
                                      "type": "string",
                                      "description": "Email of the patient",
                                  },
                                  "medic_name": {
                                      "type": "string",
                                      "description": "Name of the medic",
                                  },
                                  "appointment_datetime": {
                                      "type": "string",
                                      "description": "Date and time of the appointment",
                                  },
                                                                    
                              },
                              "required": ["patient_name", "patient_email", "medic_name", "appointment_datetime"],
                          },
                        }
                      ]
        response = requests.post(
            url="https://api.openai.com/v1/chat/completions",
            params={
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
            data=json.dumps(
                {
                    "model": model,
                    "messages": messages,
                    "functions": functions,
                    "function_call": "auto"
                }
            ),
            timeout=10,
        )
        choices = response.json()["choices"]
        answer = {
            "status_code": response.status_code,
            "text": choices[0]["message"],
        }
        return answer

    except requests.exceptions.RequestException:
        print("HTTP Request failed")
        return None


In [4]:
condensed_contexts =  [
        {
          "messages": [
              {
                "role": "user",
                "content": "Soy Alexander, mi madre se llama Sylvia, te paso los datos de ella mail: sylvia@gmail.com celu 1123343434."
              },
              {
                "role": "assistant",
                "content": "Excelente Alexander, como puedo ayudarte?"  
              },
              {
                "role": "user",
                "content": "quiero sacar un turno para ella porque no entiende nada de tecnologia y los turnos se sacan por esta via"
              },
              {
                "role": "assistant",
                "content": "No hay problema, sacamos el turno para ella, cual es el DNI de ella?"
              },
              {
                "role": "user",
                "content": "12668992"
              },
              {
                  "role": "assistant",
                  "content": "perfecto, digame que especialidad?"  
              },
              {
                  "role": "user",
                  "content": "ginecologia"
              },
              {
                  "role": "assistant",
                  "content": "Tengo turnos disponibles con el Dr. Facundo Farias para el 27 de agosto a las 10:00hs, le parece bien?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "confirmado entonces el turno para Sylvia para el 27 de agosto a las 10:00hs con el Dr. Facundo Farias. Le envio el comprobante por mail?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "Al correo de Sylvia?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "Perfecto, le envio el comprobante al mail de ella. Algo mas que pueda ayudarte?"
              },
              {
                "role": "user",
                "content": "no, muchas gracias"
              }
           ],
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "context_id": "1",
            "created_at": "20200825T12:00:00.000Z",
          },
        },     
  ]


In [5]:
db = FAISS.from_texts(texts=['notebook-14'], embedding=embeddings)
for context in condensed_contexts:
        # Join all messages in one string by a comma
        messages = ''.join([str(message) for message in context['messages']])
        # print(messages)
        # print(context['metadata'])
        extraction = gpt_system_user(
            system_message="""
              Extract every meaningful information about the user from the provided conversation.
              Ask yourself, what happened in the conversation?
              What data about the user did you learn?
              At what date and time was this data gathered?

              Create a summary of everything you learned about the user.

              Use this format:

              On [date] at [time] the [user's / user's related person] [data  learned about the user or user's related person].
              """,
            user_message=f" METADATA: ```{context['metadata']} CONVERSATION: ```{messages}``` ```",
            model="gpt-3.5-turbo",
        )
        if extraction['status_code'] != 200:
            print(f"Error: Openai API returned status code {extraction['status_code']}")
            continue
        else:
            print(f"{extraction['text']['content']} \n\n\n///////////////////////////\n\n\n")
            db.add_texts(
                texts=[extraction['text']['content']], 
                metadatas=[context['metadata']],
            )

On 20200825 at 12:00:00 the user provided the following information:
- User's name: Alexander
- User's mother's name: Sylvia
- User's mother's email: sylvia@gmail.com 
- User's mother's phone number: 1123343434
- User's mother's DNI: 12668992
- Appointment specialty: gynecology
- Appointment date and time: 27th of August at 10:00am
- Medic's name: Dr. Facundo Farias
- User confirmed the appointment and requested to send the confirmation to Sylvia's email. 


///////////////////////////





Mezclando data y fechas de dos contextos

In [6]:
user_input = "yo tengo algun turno asignado?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the personal assistant of the user.
Use the user's name in your answers.
Direct your answers to the user in the second person.
You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.



```{context}```

"""
# answer = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer = gpt_system_user(system_message=system_prompt, user_message=user_input)
# answer["text"]["content"]



 On 20200825 at 12:00:00 the user provided the following information:
- User's name: Alexander
- User's mother's name: Sylvia
- User's mother's email: sylvia@gmail.com 
- User's mother's phone number: 1123343434
- User's mother's DNI: 12668992
- Appointment specialty: gynecology
- Appointment date and time: 27th of August at 10:00am
- Medic's name: Dr. Facundo Farias
- User confirmed the appointment and requested to send the confirmation to Sylvia's email. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------




In [9]:
answer

{'status_code': 200,
 'text': {'role': 'assistant',
  'content': 'Déjame verificar tus datos para ver si tienes algún turno asignado. Por favor, espera un momento.',
  'function_call': {'name': 'create_appointment',
   'arguments': '{\n  "patient_name": "Alexander",\n  "patient_email": "sylvia@gmail.com",\n  "medic_name": "",\n  "appointment_datetime": ""\n}'}}}

Acá mezcla la data de mi madre con la mia 

```{'status_code': 200,
 'text': {'role': 'assistant',
  'content': 'Déjame verificar tus datos para ver si tienes algún turno asignado. Por favor, espera un momento.',
  'function_call': {'name': 'create_appointment',
   'arguments': '{\n  "patient_name": "Alexander",\n  "patient_email": "sylvia@gmail.com",\n  "medic_name": "",\n  "appointment_datetime": ""\n}'}}}```
   

In [10]:
user_input = "quiero un turno para mi madre con el doctor Favio Hernandez para el miercoles 8 de noviembre a las 10:00hs"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the best personal assistant of SAIA Air.
Your job is to help users in their travel needs.
Use the user's name in your answers.
You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.



```{context}```

"""
answer = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer["text"]["content"]



 On 20200825 at 12:00:00 the user provided the following information:
- User's name: Alexander
- User's mother's name: Sylvia
- User's mother's email: sylvia@gmail.com 
- User's mother's phone number: 1123343434
- User's mother's DNI: 12668992
- Appointment specialty: gynecology
- Appointment date and time: 27th of August at 10:00am
- Medic's name: Dr. Facundo Farias
- User confirmed the appointment and requested to send the confirmation to Sylvia's email. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------




In [11]:
answer

{'status_code': 200,
 'text': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'create_appointment',
   'arguments': '{\n"patient_name": "Sylvia",\n"patient_email": "sylvia@gmail.com",\n"medic_name": "Dr. Favio Hernandez",\n"appointment_datetime": "2023-11-08T10:00:00.000Z"\n}'}}}

Con el contexto apropiado, el modelo no tiene problemas hacer el llamado con los datos correctos, sin volver a pedirlos.

```
{'status_code': 200,
 'text': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'create_appointment',
   'arguments': '{\n"patient_name": "Sylvia",\n"patient_email": "sylvia@gmail.com",\n"medic_name": "Dr. Favio Hernandez",\n"appointment_datetime": "2023-11-08T10:00:00.000Z"\n}'}}}
```
